# Ý tưởng của prototype 1
Tập chung vào chạy được thuật toán thuật toán decision tree để gán mác 

Xử lý dữ liệu đơn giản để chạy được thuật toán(Xóa các hàng có giá trị null)

# Import thư viện

In [1]:
import pandas as pd
import pickle
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import json

# Xử lý dữ liệu

In [2]:
df = pd.read_csv('raw_dataset.csv')

C:\Users\ngova\AppData\Local\Temp\ipykernel_8268\1635338934.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_dataset.csv')


In [3]:
df.drop(['ID', 'Name', 'SSN', 'Month'], axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      100000 non-null  object 
 13  Num_Credit_Inquiries      98035 non-null   fl

In [5]:
df.select_dtypes('O').info()
df.select_dtypes('O').head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Customer_ID              100000 non-null  object
 1   Age                      100000 non-null  object
 2   Occupation               100000 non-null  object
 3   Annual_Income            100000 non-null  object
 4   Num_of_Loan              100000 non-null  object
 5   Type_of_Loan             88592 non-null   object
 6   Num_of_Delayed_Payment   92998 non-null   object
 7   Changed_Credit_Limit     100000 non-null  object
 8   Credit_Mix               100000 non-null  object
 9   Outstanding_Debt         100000 non-null  object
 10  Credit_History_Age       90970 non-null   object
 11  Payment_of_Min_Amount    100000 non-null  object
 12  Amount_invested_monthly  95521 non-null   object
 13  Payment_Behaviour        100000 non-null  object
 14  Monthly_Balance      

,Customer_ID,Age,Occupation,Annual_Income,Num_of_Loan,Type_of_Loan,Num_of_Delayed_Payment,Changed_Credit_Limit,Credit_Mix,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,CUS_0xd40,23,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",7,11.27,_,809.98,22 Years and 1 Months,No,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,CUS_0xd40,23,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",NaN,11.27,Good,809.98,NaN,No,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,CUS_0xd40,-500,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",7,_,Good,809.98,22 Years and 3 Months,No,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,CUS_0xd40,23,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",4,6.27,Good,809.98,22 Years and 4 Months,No,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,CUS_0xd40,23,Scientist,19114.12,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",NaN,11.27,Good,809.98,22 Years and 5 Months,No,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [6]:
df = df.applymap(lambda x: x if x is np.NaN or not isinstance(x, str) else str(x).strip('_ ,"')).replace([''], np.NaN)
df['Age'] = df.Age.astype(int)
df['Annual_Income'] = df.Annual_Income.astype(float)
df['Num_of_Loan'] = df.Num_of_Loan.astype(int)
df['Num_of_Delayed_Payment'] = df.Num_of_Delayed_Payment.astype(float)
df['Changed_Credit_Limit'] = df.Changed_Credit_Limit.astype(float)
df['Outstanding_Debt'] = df.Outstanding_Debt.astype(float)
df['Amount_invested_monthly'] = df.Amount_invested_monthly.astype(float)
df['Monthly_Balance'] = df.Monthly_Balance.astype(float)

In [7]:
def month_text_to_number(x):
    if pd.notnull(x):
        num1 = int(x.split(' ')[0])
        num2 = int(x.split(' ')[3])
        return (num1*12)+num2
    else:
        return x
df['Credit_History_Age'] = df.Credit_History_Age.apply(lambda x: month_text_to_number(x)).astype(float)

In [8]:
df.select_dtypes('O').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Customer_ID            100000 non-null  object
 1   Occupation             92938 non-null   object
 2   Type_of_Loan           88592 non-null   object
 3   Credit_Mix             79805 non-null   object
 4   Payment_of_Min_Amount  100000 non-null  object
 5   Payment_Behaviour      100000 non-null  object
 6   Credit_Score           100000 non-null  object
dtypes: object(7)
memory usage: 5.3+ MB


In [9]:
df['Occupation'].value_counts(dropna=False)

NaN              7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: Occupation, dtype: int64

In [10]:
df['Credit_Mix'].value_counts(dropna=False)

Standard    36479
Good        24337
NaN         20195
Bad         18989
Name: Credit_Mix, dtype: int64

In [11]:
df['Payment_of_Min_Amount'].value_counts(dropna=False)

Yes    52326
No     35667
NM     12007
Name: Payment_of_Min_Amount, dtype: int64

In [12]:
df['Payment_Behaviour'].value_counts(dropna=False)

Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
!@9#%8                               7600
Name: Payment_Behaviour, dtype: int64

In [13]:
df['Credit_Score'].value_counts(dropna=False)

Standard    53174
Poor        28998
Good        17828
Name: Credit_Score, dtype: int64

In [14]:
df['Occupation'] = df.Occupation.apply(lambda x: 'Unknown' if pd.isnull(x) else x)
df['Credit_Mix'] = df.Credit_Mix.apply(lambda x: 'Unknown' if pd.isnull(x) else x)
df['Payment_of_Min_Amount'] = df.Payment_of_Min_Amount.apply(lambda x: 'Unknown' if x == 'NM' else x)
df['Payment_Behaviour'] = df.Payment_Behaviour.apply(lambda x: 'Unknown' if x == '!@9#%8' else x)

In [15]:
occupations = list(df.Occupation.astype('category').cat.categories)
occupations.remove('Unknown')
occupations.append('Unknown')
df['Occupation'] = df.Occupation.apply(lambda x: occupations.index(x))

payment_min_amounts = list(df.Payment_of_Min_Amount.astype('category').cat.categories)
payment_min_amounts.remove('Unknown')
payment_min_amounts.append('Unknown')
df['Payment_of_Min_Amount'] = df.Payment_of_Min_Amount.apply(lambda x: payment_min_amounts.index(x))

payment_behaviours = list(df.Payment_Behaviour.astype('category').cat.categories)
payment_behaviours.remove('Unknown')
payment_behaviours.append('Unknown')
df['Payment_Behaviour'] = df.Payment_Behaviour.apply(lambda x: payment_behaviours.index(x))

In [16]:
credit_mixs = ['Bad', 'Standard', 'Good', 'Unknown']
df['Credit_Mix'] = df.Credit_Mix.apply(lambda x: credit_mixs.index(x))

credit_scores = ['Poor', 'Standard', 'Good']
df['Credit_Score'] = df.Credit_Score.apply(lambda x: credit_scores.index(x))

In [17]:
df['Type_of_Loan'] = df['Type_of_Loan'].str.replace(', and', ',')
df['Num_of_Loan'] = df['Type_of_Loan'].map(lambda x: len(x.split(', ')) if not pd.isnull(x) else 0)
loans_set = list(set([y for x in df['Type_of_Loan'].dropna() for y in x.split(', ')]))
for item in loans_set:
    df[item] = df['Type_of_Loan'].map(lambda x: x.count(item) if not pd.isnull(x) else 0)
df.drop("Type_of_Loan", axis=1, inplace=True)

In [18]:
df.dropna(inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65143 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               65143 non-null  object 
 1   Age                       65143 non-null  int32  
 2   Occupation                65143 non-null  int64  
 3   Annual_Income             65143 non-null  float64
 4   Monthly_Inhand_Salary     65143 non-null  float64
 5   Num_Bank_Accounts         65143 non-null  int64  
 6   Num_Credit_Card           65143 non-null  int64  
 7   Interest_Rate             65143 non-null  int64  
 8   Num_of_Loan               65143 non-null  int64  
 9   Delay_from_due_date       65143 non-null  int64  
 10  Num_of_Delayed_Payment    65143 non-null  float64
 11  Changed_Credit_Limit      65143 non-null  float64
 12  Num_Credit_Inquiries      65143 non-null  float64
 13  Credit_Mix                65143 non-null  int64  
 14  Outsta

In [20]:
df.drop("Customer_ID", axis=1, inplace=True)

In [21]:
df.to_csv('dataset.csv', index=False)

# Train from clean data

In [22]:
df = pd.read_csv("./dataset.csv", sep = "," , encoding = "utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65143 entries, 0 to 65142
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       65143 non-null  int64  
 1   Occupation                65143 non-null  int64  
 2   Annual_Income             65143 non-null  float64
 3   Monthly_Inhand_Salary     65143 non-null  float64
 4   Num_Bank_Accounts         65143 non-null  int64  
 5   Num_Credit_Card           65143 non-null  int64  
 6   Interest_Rate             65143 non-null  int64  
 7   Num_of_Loan               65143 non-null  int64  
 8   Delay_from_due_date       65143 non-null  int64  
 9   Num_of_Delayed_Payment    65143 non-null  float64
 10  Changed_Credit_Limit      65143 non-null  float64
 11  Num_Credit_Inquiries      65143 non-null  float64
 12  Credit_Mix                65143 non-null  int64  
 13  Outstanding_Debt          65143 non-null  float64
 14  Credit

In [23]:
labels = ['Poor', 'Standard', 'Good']

In [24]:
X = df.drop(['Credit_Score'], axis=1)
y = df['Credit_Score']
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=36)

print(X_train.shape, y_train.shape)

(52114, 30) (52114,)


In [25]:
unique, counts = np.unique(y_train, return_counts=True)
pfunct = np.vectorize(lambda x: x/len(y_train))
pfunct(counts)

array([0.28949994, 0.53377211, 0.17672794])

In [26]:
unique, counts = np.unique(y_test, return_counts=True)
pfunct = np.vectorize(lambda x: x/len(y_test))
pfunct(counts)

array([0.28950802, 0.53373244, 0.17675954])

# Sử dụng Decision Tree

In [27]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(pd.DataFrame(metrics.confusion_matrix(y_test, y_pred, labels=np.unique(y_test)), 
             index=['True ' + x for x in labels], 
             columns=['Predict ' + x for x in labels]))
print(metrics.classification_report(y_test, y_pred, target_names=labels))

               Predict Poor  Predict Standard  Predict Good
True Poor              2537              1080           155
True Standard          1174              4929           851
True Good               176               807          1320
              precision    recall  f1-score   support

        Poor       0.65      0.67      0.66      3772
    Standard       0.72      0.71      0.72      6954
        Good       0.57      0.57      0.57      2303

    accuracy                           0.67     13029
   macro avg       0.65      0.65      0.65     13029
weighted avg       0.68      0.67      0.67     13029



Độ chính xác khi sử dụng DecisionTree là khoảng 67%

In [28]:
def cross_val_metrics(clf, X, y, cv=5):
    accuracy = []
    f1 = []
    precision = []
    recall = []
    kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=36)
    for train_idx, test_idx in kf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy.append(metrics.accuracy_score(y_test, y_pred))
        precision.append(metrics.precision_score(y_test, y_pred, average=None))
        recall.append(metrics.recall_score(y_test, y_pred, average=None))
        f1.append(metrics.f1_score(y_test, y_pred, average=None))
    print("accuracy", np.mean(accuracy))
    print("precision", np.mean(precision, axis=0))
    print("recall",np.mean(recall, axis=0))
    print("f1",np.mean(f1, axis=0))

In [29]:
cross_val_metrics(clf, X, y)

accuracy 0.6766191066700761
precision [0.66163695 0.7188689  0.57508937]
recall [0.66159406 0.71643026 0.58099582]
f1 [0.66159379 0.71763825 0.57799862]


# Nhận xét về prototype 1

## Điểm tốt
+ Đã thực hiện shuffle dữ liệu trước khi phân thành train và test, tránh tình trạng dữ liệu bị biased(VD Train chỉ có dữ liệu mác good, test chỉ có dữ liệu mác standard) nhưng giữ được tỷ lệ các mác trong tập train và test.

## Điểm xấu
+ Quá trình phân tích và xử lý nhiễu mất mát nhiều dữ liệu(Có nhiều hàng dữ liệu dù chỉ trống một trường nhưng vẫn bị loại bỏ).
+ Quá trình phân tích và xử lý dữ liệu chưa tính toán đến các dị điểm trong dữ liệu.
+ Kết quả thu được chưa thực sự tốt ở nhãn Good (một phần do số lượng các bản ghi mang nhãn Good thấp hơn đáng kể so với các nhãn còn lại)